# Problema 3 - Análisis Datos Youtube - Archivo .py

1. Descargar un archivo .zip mediante código del siguiente url (https://netsg.cs.sfu.ca/youtubedata/) (recomiendo descargar el archivo 0333.zip que es menos pesado)
2. Descomprimir los datos en una carpeta que genere y leer mediante pandas alguno de los archivos en esta. (observar que no es necesario en un primer momento leer los datos con un nombre de columna especifico)

    - Los nombres de columna pueden ser puestos posteriormente
    - El separador de columna es <code>\t</code>
    - Se colocan los nombres de columnas y descripción asociada para su intermetación. Ejemplo columna1 sera VideoID ... 
    

3. Procesar los datos según: 
    - Nos quedaremos con las columnas: VideoID, edad, catgoria, views, rate.
    - Realizar un filtrado básico a los datos. Ejemplo solo seleccionar cierto grupo de categorias

4. Procesamiento en Mongo Db
    - Exportar los datos a mongo DB 
    - Crear 2 graficos con los datos 
    - Compartir link donde encontrar los datos 




| Nombre de la Columna | Descripción                                                                                                 |
|----------------------|-------------------------------------------------------------------------------------------------------------|
| `video ID`           | Una cadena de 11 dígitos, la cual es única                                                                |
| `uploader`           | Una cadena con el nombre de usuario del cargador del video                                                  |
| `age`                | Un número entero que representa los días transcurridos desde la fecha en que se subió el video hasta el 15 de febrero de 2007 (fecha de creación de YouTube) |
| `category`           | Una cadena que indica la categoría del video elegida por el cargador                                       |
| `length`             | Un número entero que representa la duración del video en minutos                                            |
| `views`              | Un número entero que representa el número de visualizaciones del video                                      |
| `rate`               | Un número flotante que indica la calificación del video                                                      |
| `ratings`            | Un número entero que representa el número de calificaciones recibidas por el video                          |
| `comments`           | Un número entero que indica el número de comentarios en el video                                            |
| `related IDs`        | Hasta 20 cadenas de texto con los IDs de videos relacionados                                                |


In [ ]:
import requests
import zipfile
import pandas as pd
import pymongo
import matplotlib.pyplot as plt
url = "https://netsg.cs.sfu.ca/youtubedata/0333.zip"
response = requests.get(url, stream=True)

with open("0333.zip", 'wb') as out_file:
    for chunk in response.iter_content(chunk_size=1024):
        if chunk:
            out_file.write(chunk)
with zipfile.ZipFile("0333.zip", 'r') as zip_ref:
    zip_ref.extractall("youtube_data")
df = pd.read_csv("youtube_data/data.csv", sep=",")
df = df[['VideoID', 'age', 'category', 'views', 'rate']]
df = df[df['category'] == 'Music']
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["youtube_data"]
collection = db["videos"]
collection.insert_many(df.to_dict('records'))
plt.hist(df['views'], bins=20)
plt.xlabel('Número de vistas')
plt.ylabel('Frecuencia')
plt.title('Distribución del número de vistas de videos musicales')
plt.show()

# Exportar datos a un archivo CSV (opcional)
df.to_csv('videos_musicales.csv', index=False)
print("Datos exportados a videos_musicales.csv")